In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVR

from sklearn.preprocessing import OneHotEncoder, StandardScaler
OHE = OneHotEncoder(sparse=False)
pd.set_option('display.max_rows', 30)

In [2]:
# 月日如果用起来要one-hot，之后产生的feature就太多了而且感觉没必要，可能考虑is_special更合适
# 但是前后依赖这里怎么办呢？
# train[['month','day']] = train.date.str.split('-', expand=True)
def get_x_y(df):
    data = df.drop(columns=['Unnamed: 0', 'route', 'calculated_day', 'date'])
    ohe = OHE.fit_transform(data[['hr', 'weekday']])
    ohe_df = pd.DataFrame(ohe, columns= OHE.get_feature_names(input_features=['hr', 'weekday']))
    data = pd.concat([data, ohe_df],axis = 1)
    data = data.drop(columns=['hr', 'weekday'])
    x = data.drop(columns=['label'])
    y = data.label
    return x, y

In [3]:
def mape_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))


def scoring(reg, x, y):
    pred = reg.predict(x)
    return -mape_error(pred, y)

In [6]:
#A_2
file_path = 'A_2.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(valid)

gdbt = SVR().fit(train_x, train_y)
print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

# gbdt的结果
# 0.09230943513239677
# 0.1381704938850841

0.16987696954765566
0.11482559452768365
          label          0
0     75.086667  70.843777
1    148.053333  73.952812
2    109.863333  76.770184
3     74.633333  76.941981
4     88.636667  76.131967
5     65.770000  71.751233
6    173.260000  76.744904
7     88.680000  74.391220
8    140.553333  76.157766
9     84.563333  73.067956
10   107.946667  68.306587
11    93.943333  68.952712
12    68.176667  70.469619
13   108.560000  77.161129
14    85.526667  77.168508
..          ...        ...
276   57.766667  71.073710
277   90.673333  68.548808
278   76.296667  70.462380
279   89.176667  70.581627
280   59.073333  67.696270
281   89.446667  76.475414
282   56.376667  65.218425
283   53.956667  63.969709
284   46.476667  64.840840
285   66.246667  67.193294
286   63.916667  68.380762
287   68.750000  68.619857
288   58.106667  65.357271
289   58.666667  65.434214
290   63.286667  71.415535

[291 rows x 2 columns]
         label          0
0    80.126667  68.320779
1    68.713333  66.2

In [8]:
#A_3
file_path = 'A_3.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(valid)

gdbt = SVR().fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

# gbdt
# 0.12228678785309767
# 0.1471140890831642

0.17297326402407368
0.13899183024742304
          label           0
0    146.513333  119.048969
1    203.340000  133.623329
2    168.586667  140.211589
3    139.636667  129.071784
4    108.560000  125.836135
5    112.563333  112.907590
6    158.526667  133.291408
7    132.176667  125.200628
8    123.523333  134.152954
9    159.600000  135.281943
10   172.003333  134.176524
11   119.686667  118.741526
12   123.146667  120.691276
13   177.236667  128.777828
14   172.870000  123.658294
..          ...         ...
272  122.283333  115.493917
273  109.413333  118.319054
274  120.263333  129.614964
275  120.730000  117.335166
276  214.430000  123.486997
277  122.100000  122.901974
278  100.010000  113.625556
279  108.500000  117.583981
280  103.200000  118.182443
281  121.746667  123.678381
282  115.176667  133.717620
283  109.960000  115.082291
284  123.546667  115.648268
285  105.820000  116.709003
286  120.643333  122.627673

[287 rows x 2 columns]
         label           0
0   124.72000

In [9]:
#B_1
file_path = 'B_1.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)


train_p5= pd.read_csv('5min_shifted_with_10.1_B_1.csv')
train_m5= pd.read_csv('-5min_shifted_with_10.1_B_1.csv')


valid_index=[]

cal_day=list(train_p5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_p5_used=train_p5.drop(valid_index)
train_p5_used.reset_index(drop=True, inplace=True)


valid_index=[]

cal_day=list(train_m5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_m5_used=train_m5.drop(valid_index)
train_m5_used.reset_index(drop=True, inplace=True)


train1 = pd.concat([train,train_p5_used])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(valid)



gdbt = SVR().fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

# gbdt
# 0.2405223397424579
# 0.1299667129187069

0.23235859315856172
0.12794120340420484
          label           0
0    100.320000  118.449212
1    212.310000  119.999872
2    129.945000  117.890299
3    118.616667  117.457230
4    118.785000  116.843211
5    101.216667  119.772737
6    179.000000  117.995105
7     23.100000  118.470615
8    142.260000  119.892331
9    102.365000  117.395325
10    88.320000  116.749572
11    97.943333  116.996571
12   101.250000  114.908938
13   102.785000  116.706990
14   112.445000  116.196441
..          ...         ...
452  289.906667  121.114602
453  129.943333  120.187087
454  297.500000  118.387338
455  122.950000  119.020773
456  137.385000  119.211592
457  129.946667  124.488760
458  129.440000  116.979582
459  108.876667  117.381385
460  113.066667  119.278442
461  126.573333  119.009900
462  121.953333  121.433857
463  110.900000  117.193546
464  144.060000  118.162292
465  130.540000  116.979282
466  215.270000  121.070373

[467 rows x 2 columns]
         label           0
0   132.93000

In [11]:
train1

,Unnamed: 0,label,route,calculated_day,date,hr,weekday,110,123,107,...,precipitation,105,100,111,103,116,101,121,106,113
0,0,146.513333,A_3,236,7-19,8,1,13.050750,7.050500,4.841250,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,203.340000,A_3,237,7-20,8,2,15.771250,8.656250,5.604000,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,168.586667,A_3,238,7-21,8,3,16.371636,9.418364,6.588000,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,139.636667,A_3,239,7-22,8,4,13.753182,7.978409,5.906364,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,108.560000,A_3,240,7-23,8,5,13.491200,7.297800,5.067000,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,284,110.900000,B_1,316,10-6,18,3,NaN,NaN,NaN,...,0.0,21.561176,8.350909,23.527826,4.491957,16.924091,10.487273,14.486364,2.177273,18.041364
525,285,144.060000,B_1,317,10-7,18,4,NaN,NaN,NaN,...,2.0,15.984000,8.490227,29.352264,6.520189,19.960000,13.539545,14.991818,1.774545,20.460000
526,286,130.540000,B_1,318,10-8,18,5,NaN,NaN,NaN,...,0.0,11.991111,6.916111,18.296765,4.507353,15.415263,8.717368,24.152632,1.964737,22.461053
527,287,215.270000,B_1,319,10-9,18,6,NaN,NaN,NaN,...,0.0,29.550000,7.697917,29.640769,6.865641,17.461333,8.989333,18.533333,3.262667,28.375333


In [10]:
#B_3
file_path = 'B_3.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(valid)

gdbt = SVR().fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

# gbdt
# 0.10536970076864392
# 0.19815368489358295

0.1905780564218508
0.14713125412442687
          label           0
0    147.723333  116.295668
1    148.253333  127.589987
2    107.110000  115.832310
3    155.416667  115.042671
4    103.586667  115.929656
5     67.636667  110.622676
6    115.960000  117.266421
7    128.463333  111.444676
8    109.366667  109.518021
9    139.310000  123.436308
10   164.433333  129.100136
11   141.243333  113.660828
12   154.413333  112.585696
13   142.190000  125.230910
14   208.746667  116.153014
..          ...         ...
274  109.420000  109.295958
275  132.630000  115.001683
276   97.790000  116.675612
277  134.226667  112.769483
278  596.750000  116.577331
279  110.676667  123.221075
280   81.350000  106.644458
281  102.563333  109.321578
282  125.633333  112.377678
283  107.600000  113.127178
284  103.366667  116.203832
285   90.780000  108.614765
286  111.426667  110.029290
287  104.705000  109.445450
288  121.530000  112.750801

[289 rows x 2 columns]
         label           0
0   102.530000

In [11]:
#C_1
file_path = 'C_1.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)


train_p5= pd.read_csv('5min_shifted_C_1.csv')
train_m5= pd.read_csv('-5min_shifted_C_1.csv')


valid_index=[]

cal_day=list(train_p5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_p5_used=train_p5.drop(valid_index)
train_p5_used.reset_index(drop=True, inplace=True)


valid_index=[]

cal_day=list(train_m5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_m5_used=train_m5.drop(valid_index)
train_m5_used.reset_index(drop=True, inplace=True)


train1 = pd.concat([train,train_p5_used])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(valid)



gdbt = SVR().fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

# gbdt
# 0.10830155005055248
# 0.1390390318323988

0.1718242795134562
0.15175864453383212
          label           0
0    140.245000  176.268837
1    231.923333  175.718095
2    146.070000  173.599411
3    190.573333  172.940955
4    207.370000  173.468007
5    156.363333  172.532851
6    152.426667  177.258006
7    189.870000  175.942664
8    130.555000  173.486033
9    173.980000  173.260912
10   138.240000  175.188054
11   243.325000  173.601719
12   124.545000  173.649847
13   171.280000  174.683435
14   150.573333  173.026700
..          ...         ...
416  238.696667  174.585042
417  232.180000  175.072259
418  365.560000  181.959127
419  373.416667  181.585854
420  157.866667  173.881693
421  293.980000  177.818165
422  353.680000  177.781040
423  208.270000  175.195601
424  205.230000  173.616257
425  337.190000  177.406474
426  252.450000  174.815226
427  268.755000  174.758587
428  249.116667  181.753318
429  272.230000  172.670725
430  238.060000  174.912493

[431 rows x 2 columns]
         label           0
0   159.553333

In [12]:
#C_3
file_path = 'C_3.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)


train_p5= pd.read_csv('5min_shifted_C_3.csv')
train_m5= pd.read_csv('-5min_shifted_C_3.csv')


valid_index=[]

cal_day=list(train_p5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_p5_used=train_p5.drop(valid_index)
train_p5_used.reset_index(drop=True, inplace=True)


valid_index=[]

cal_day=list(train_m5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_m5_used=train_m5.drop(valid_index)
train_m5_used.reset_index(drop=True, inplace=True)


train1 = pd.concat([train,train_p5_used])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(valid)



gdbt = SVR().fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

0.21047501210223693
0.18192500975375567
          label           0
0    132.050000  187.203978
1    219.150000  194.833718
2    166.150000  187.648907
3    161.786667  187.334001
4    207.950000  184.139266
5    266.680000  188.332090
6    197.346667  184.518387
7    223.800000  183.106279
8    294.460000  190.153347
9    129.590000  194.579994
10   164.520000  184.782399
11   197.700000  184.676719
12   227.955000  194.771443
13   300.616667  187.027329
14   253.250000  188.723827
..          ...         ...
383  195.595000  184.936966
384  165.435000  183.918213
385  160.405000  189.993695
386  314.240000  191.318044
387  171.436667  183.097874
388  307.575000  188.726922
389  209.406667  190.668326
390  188.146667  182.222536
391  182.946667  185.343643
392  173.840000  188.774223
393  307.935000  183.000988
394  189.050000  186.317221
395  245.666667  189.683022
396  141.100000  180.875796
397  182.425000  182.993393

[398 rows x 2 columns]
         label           0
0   198.08000